In [ ]:
import cv2 as cv
import json
import numpy as np
import os
from matplotlib import pyplot as plt
from PIL import Image

# expects fretboard dataset to be in data folder on same level as tabs_generator folder
train_str = "test" # change to train or test to perform on respective data
dataset_path = "../data/fretboard_dataset/"
images_path = f"../data/fretboard_dataset/fretboard_frames_{train_str}/"
image_fns = os.listdir(images_path)
json_fn = os.path.join(dataset_path, f"fretboard_labels_{train_str}_vgg.json")
f = open(json_fn)
data = json.load(f) # data is dict of json contents

for fn in image_fns:
    img_path = os.path.join(images_path, fn)
    out_path = os.path.join(images_path, "annotated_"+fn)
    img = np.asarray(Image.open(img_path)) # loads images w/ range of 0:255
    x_list = data[fn]["regions"]["0"]["shape_attributes"]["all_points_x"] # list of rectangle coordinates
    y_list = data[fn]["regions"]["0"]["shape_attributes"]["all_points_y"]
    x_list.pop(-1) # remove duplicate of first coordinate from end of lists
    y_list.pop(-1)
    poly_corners = np.asarray([list(zip(x_list, y_list))], dtype=np.int32) # repackage into np array w/ correct dtype
    new_img = np.zeros((img.shape[0], img.shape[1]), np.uint8) # create empty binary image mask
    new_img = cv.fillPoly(new_img, poly_corners, 255) # fill polygon of rectangle coordinates
    Image.fromarray(new_img).save(out_path) # save output image